## 3.1. Write out the equation for trigram probability estimation

$P(w_n|w_{n-2}w_{n-1}) = \frac{C(w_{n-2}w_{n-1}w_n)}{C(w_{n-2}w_{n-1})}$

Now write out all the non-zero trigram probabilities for the *I am Sam* corpus

    <s> I am Sam </s>
    <s> Sam I am </s>
    <s> I do not like green eggs and ham </s>

In [ ]:
class TrigramModel():
  def __init__(self, corpus):
    # self.unigrams = {}
    self.bigrams = {}
    self.trigrams = {}

    for s in corpus:
      words = s.lower().split()
      # # appending unigrams
      # for w in words:
      #   if w not in self.unigrams.keys():
      #     self.unigrams[w] = 1
      #   else:
      #     self.unigrams[w] += 1
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in self.bigrams:
          self.bigrams[cur_bigram] = 1
        else:
          self.bigrams[cur_bigram] += 1
        cntr += 1
      # appending trigrams
      cntr = 0
      for i in range(len(words)-2):
        cur_trigram = (words[cntr] + ' ' + words[cntr+1] + ' ' + words[cntr+2])
        if cur_trigram not in self.trigrams:
          self.trigrams[cur_trigram] = 1
        else:
          self.trigrams[cur_trigram] += 1
        cntr += 1

  # def add1smoothing_proba(self, s):
  #   s = s.lower()
  #   words = s.split()
  #   return (self.trigrams[s] + 1) / (self.unigrams[words[0]]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.trigrams[s]/self.bigrams[words[0] + ' ' + words[1]]

In [ ]:
corpus = [
    '<s> I am Sam </s>',
    '<s> Sam I am </s>',
    '<s> I do not like green eggs and ham </s>',
]

model = TrigramModel(corpus=corpus)

In [ ]:
print('Answer: ')
for t in model.trigrams.keys():
  print(f'P({t}) = {model.unsmoothed_proba(t)}')

Answer: 
P(<s> i am) = 0.5
P(i am sam) = 0.5
P(am sam </s>) = 1.0
P(<s> sam i) = 1.0
P(sam i am) = 1.0
P(i am </s>) = 0.5
P(<s> i do) = 0.5
P(i do not) = 1.0
P(do not like) = 1.0
P(not like green) = 1.0
P(like green eggs) = 1.0
P(green eggs and) = 1.0
P(eggs and ham) = 1.0
P(and ham </s>) = 1.0


## 3.2 Calculate the probability of the sentence *i want chinese food*. Give two probabilities -- unsmoothed and an add-one probability. Assume the additional add-1 smoothed probabilities $P(i|<s>)=0.19$ and $P(</s>|food)=0.40$.

In [ ]:
sentence = '<s> I want chinese food <\s>'
unsmoothed_proba = 0.25*0.33*0.0065*0.52*0.68
addone_proba = 0.19*0.21*0.0029*0.52*0.4
print('Answer\n',\
      'Unsmoothed probability:', "{:.5f}".format(unsmoothed_proba), '\n',\
      'Add-one probability:', "{:.7f}".format(addone_proba))

Answer
 Unsmoothed probability: 0.00019 
 Add-one probability: 0.0000241


## 3.3 Which of the two probabilities you computed in the previous exercise is higher, unsmoothed or smoothed? Explain why.

Answer:

The unsmoothed probability is higher because we did not give away any bit of it to the events we have not seen, unlike smoothed probability which is what's left after we've 'shaved off' some of the probability mass.

## 3.4 We are given the following corpus:

    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',

Using a bigram language model with add-one smoothing, what is P(Sam|am)? Include \<s> and \<\s> in your counts just like any other token.

In [ ]:
class BigramModel():
  def __init__(self, corpus):
    self.unigrams = {}
    self.bigrams = {}

    for s in corpus:
      words = s.split()
      # appending unigrams
      for w in words:
        if w not in self.unigrams.keys():
          self.unigrams[w] = 1
        else:
          self.unigrams[w] += 1
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in self.bigrams:
          self.bigrams[cur_bigram] = 1
        else:
          self.bigrams[cur_bigram] += 1
        cntr += 1

  def add1smoothing_proba(self, s):
    s = s.lower()
    words = s.split()
    return (self.bigrams[s] + 1) / (self.unigrams[words[0]]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.bigrams[s]/self.unigrams[words[0]]

In [ ]:
corpus = [
    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',
]

model = BigramModel(corpus=corpus)
print('Answer: ', round(model.add1smoothing_proba('am Sam'), 3))

Answer:  0.214


## 3.5 Suppose we didn't use the end-symbol \<\s>. Train an unsmoothed bigram grammar on the following training corpus without using the end-symbol \<\s>:

    '<s> a b',
    '<s> b b',
    '<s> b a',
    '<s> a a'

In [ ]:
corpus = [
    '<s> a b',
    '<s> b b',
    '<s> b a',
    '<s> a a'
]

model = BigramModel(corpus=corpus)
model.unsmoothed_proba('a a') + model.unsmoothed_proba('a b') + model.unsmoothed_proba('b b') + model.unsmoothed_proba('b a')

1.0

## 3.6 Suppose we train a trigram language model with add-one smoothing on a given corpus. The corpus contains V word types. Express a formula for estimating $P(w3,w1,w2)$, where $w3$ is a word which follows the bigram $(w1,w2)$, in terms of various n-gram counts and V.

$P_{add-one} (w3|w1w2) = \frac{C(w1w2w3)+1}{C(w1w2)+V_2}$

, where $V_2$ is the number of unique bigrams in the corpus

## 3.7 We are given the following corpus, modified from the one in the chapter:
    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',

If we use linear interpolation smoothing between a maximum-likelihood bigram
model and a maximum-likelihood unigram model with $\lambda_1 = \frac{1}{2}$ and $\lambda_2 = \frac{1}{2}$ , what is $P(Sam|am)$? Include \<s> and </s> in your counts just like any other token.

Answer:

$\hat P(Sam|am) = \lambda_1*P(Sam) + \lambda_2*P(am|Sam)$

In [ ]:
lambda_1 = 0.5
lambda_2 = 0.5
p_sam = 4/25
p_amsam = 2/3
p_interpolation = lambda_1*p_sam + lambda_2*p_amsam
print('Answer: ', p_interpolation)

Answer:  0.41333333333333333


## 3.8 Write a program to compute unsmoothed unigrams and bigrams.

In [ ]:
import random

class NgramExtractor():
  def __init__(self, filename):
    self.filename = filename
    self.corpus = self.read_text()
    self.unigrams = self.extract_unigrams()
    self.bigrams = self.extract_bigrams()

  def read_text(self):
    with open(self.filename) as f:
      lines = f.readlines()

    import re
    corpus = []
    for line in lines:
      line = line.replace('\n', '')
      sentences = re.split('[.?!]\s*', line)
      for s in sentences:
        if s != '':
          corpus.extend(['<s> ' + s.lower().replace(',', '') + ' <\s>'])
    return corpus

  def extract_unigrams(self):

    lst = {}
    for s in self.corpus:
      words = s.split()
      # appending unigrams
      for w in words:
        if w not in lst.keys():
          lst[w] = 1
        else:
          lst[w] += 1
    total = sum(lst.values())
    for index, value in lst.items():
      lst[index] = (value, value/total)
    return lst

  def extract_bigrams(self):
    lst = {}
    for s in self.corpus:
      words = s.split()
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in lst:
          lst[cur_bigram] = 1
        else:
          lst[cur_bigram] += 1
        cntr += 1
    total = sum(lst.values())
    for index, value in lst.items():
      lst[index] = (value, value/total)
    return lst

  def add1smoothing_proba(self, s):
    s = s.lower()
    words = s.split()
    return (self.bigrams[s][0] + 1) / (self.unigrams[words[0]][0]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.bigrams[s][0]/self.unigrams[words[0]][0]

  def generate_random_sentence(self, length):
    words = list(self.unigrams.keys())
    words.remove('<s>')
    words.remove('<\s>')
    return ' '.join(random.sample(words, length))

  def calculate_ppl(self, sentence):
    ppl = 1
    sentence = ['<s>'] + sentence.strip().split() + ['<\s>']
    for w in sentence:
      ppl = self.unigrams[w][1]
    return pow(ppl, -1/len(sentence))

## 3.9 Run your n-gram program on two different small corpora of your choice (you might use email text or newsgroups). Now compare the statistics of the two corpora. What are the differences in the most common unigrams between the two? How about interesting differences in bigrams?

In [ ]:
filename = 'MurderOfTheUniverse.txt' #@param
model1 = NgramExtractor(filename)
model1.corpus[:10]

['<s> as soon as the dust settles you can see <\\s>',
 '<s> a new world in place of where the old one had been <\\s>',
 '<s> your skin is crawling with dry crusted mud <\\s>',
 '<s> and your naked feet are wet in a pool of blood <\\s>',
 '<s> and the whistle of the wind in your ears is so loud <\\s>',
 '<s> that your memories have blown up in a mushroom cloud <\\s>',
 '<s> and as your eyes accommodate <\\s>',
 '<s> there appears by the meadow <\\s>',
 '<s> a brute like a bear with a long dark shadow <\\s>',
 '<s> and you violently shake over what you have seen <\\s>']

In [ ]:
import pandas as pd
motu = pd.Series(model1.unigrams).sort_values(ascending=False).drop(index=['<s>', '<\s>'])
motu[:50]

i            (192, 0.040387042490534285)
the          (161, 0.033866217921750104)
and          (121, 0.025452250736222128)
to            (80, 0.016827934371055953)
you           (73, 0.015355490113588556)
a              (68, 0.01430374421539756)
of            (58, 0.012200252419015565)
altered       (54, 0.011358855700462769)
my             (52, 0.01093815734118637)
balrog        (47, 0.009886411442995372)
feel          (46, 0.009676062263357174)
me            (45, 0.009465713083718973)
beast         (44, 0.009255363904080775)
don't         (38, 0.007993268826251577)
am            (37, 0.007782919646613378)
in            (37, 0.007782919646613378)
is            (37, 0.007782919646613378)
your          (33, 0.006941522928060581)
see           (31, 0.006520824568784182)
it            (31, 0.006520824568784182)
vomit         (29, 0.006100126209507783)
oh            (25, 0.005258729490954985)
no            (24, 0.005048380311316786)
alter         (22, 0.004627681952040387)
that          (2

In [ ]:
filename = 'NonagonInfinity.txt' #@param
model2 = NgramExtractor(filename)
model2.corpus[:10]

['<s> nonagon infinity opens the door <\\s>',
 '<s> nonagon infinity opens the door <\\s>',
 '<s> wait for the answer to open the door <\\s>',
 '<s> nonagon infinity opens the door <\\s>',
 '<s> one two three <\\s>',
 '<s> loosen up <\\s>',
 '<s> time to drop <\\s>',
 '<s> fuck shit up <\\s>',
 "<s> don't forget about it <\\s>",
 "<s> my coffin's all i see <\\s>"]

In [ ]:
nonagon = pd.Series(model2.unigrams).sort_values(ascending=False).drop(index=['<s>', '<\s>'])
nonagon[:50]

the           (109, 0.04344360302909526)
a             (51, 0.020326823435631726)
i             (46, 0.018333997608609008)
my             (36, 0.01434834595456357)
it's          (35, 0.013949780789159028)
beat           (33, 0.01315265045834994)
to            (30, 0.011956954962136309)
i'm           (27, 0.010761259465922678)
door           (25, 0.00996412913511359)
wasp           (25, 0.00996412913511359)
of            (24, 0.009565563969709047)
gamma         (24, 0.009565563969709047)
is            (23, 0.009166998804304504)
knife          (22, 0.00876843363889996)
infinity       (22, 0.00876843363889996)
nonagon        (22, 0.00876843363889996)
only          (21, 0.008369868473495417)
fig           (21, 0.008369868473495417)
and           (20, 0.007971303308090873)
it            (18, 0.007174172977281785)
you           (18, 0.007174172977281785)
miss          (18, 0.007174172977281785)
for           (17, 0.006775607811877242)
mr            (16, 0.006377042646472699)
once          (1

In [ ]:
cntr = 0
print('Answer: ')
for t in model.bigrams.keys():
  if cntr < 10:
    # if t.startswith('you '):
    print(f'P({t}) = {model.unsmoothed_proba(t)}')
    cntr +=1

Answer: 
P(<s> nonagon) = 0.05250596658711217
P(nonagon infinity) = 1.0
P(infinity opens) = 0.6363636363636364
P(opens the) = 0.9333333333333333
P(the door) = 0.22018348623853212
P(door <\s>) = 0.8
P(<s> wait) = 0.011933174224343675
P(wait for) = 1.0
P(for the) = 0.47058823529411764
P(the answer) = 0.045871559633027525


Answer:

Not really the best or even suitable material for this excercise but still better than nothing. Initially was planning to use happy new year messages published by some independent russian media in 2020, 2021 vs. 2022, 2023, because i thought they would show the shift of the focus from the coronavirus in 2020-2021 to the war in 2022-2023. But sadly i didn't find any media that published such messages for 4 years in a row. So instead settled for lyrics from two King Gizzard and the Lizzard Wizard albums -- Murder of the Universe and Nonagon Infinity.

Both showed the highest frequencies for the articles 'the' and 'a' and pronouns like 'i', 'me' and 'my'. In the first album, among most used words are those associated with repulsiveness and evil, such as 'beast', 'balrog', 'vomit', 'pain', 'coffin' etc. which goes hand in hand with the album's narrative.

## 3.10 Add an option to your program to generate random sentences.

In [ ]:
# generate a random sentence of 10 words
model1.generate_random_sentence(10)

'swathe deepest heave sabre life lugubrious slip a rotten soggy.'

In [ ]:
# generate a random sentence of 10 words
model2.generate_random_sentence(10)

'limbs dream dueling rubber obliteration while winged of aflame fig.'

## 3.11 Add an option to your program to compute the perplexity of a test set.

In [ ]:
# calculate unigram probability of a random sentence
model1.calculate_ppl('swathe deepest heave sabre life lugubrious slip a rotten soggy')

1.1765021002165172

## 3.12 You are given a training set of 100 numbers that consists of 91 zeros and 1 each of the other digits 1-9. Now we see the following test set: 0 0 0 0 0 3 0 0 0 0. What is the unigram perplexity?

In [ ]:
pow((0.91**9)*0.09, -1/10)

1.384964035825701